# Imports

In [ ]:
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from hbt_tools import saba 

# Functions

In [ ]:
def read_messstelle(file_path, col_Q):
    date_format = '%d.%m.%Y %H:%M:%S'
    data = pd.read_csv(file_path, sep=';', encoding='UTF-8', usecols=['Datum', 'Uhrzeit', col_Q],
        parse_dates=[[0,1]], date_format=date_format, decimal=',')
    data.set_index('Datum_Uhrzeit', inplace=True)
    data = data.astype(float, copy=True)

    return data
#-----------------------------------------------------------------------------------------------------------------------

def read_regen(file_path):
    date_format = '%d.%m.%Y %H:%M:%S'
    data = pd.read_csv(file_path, sep=';', encoding='UTF-8', usecols=['Datum', 'Uhrzeit', 'Niederschlag [mm]'],
            parse_dates=[[0,1]], date_format=date_format, decimal=',')
    data.set_index('Datum_Uhrzeit', inplace=True)

    return data

# Parameters

In [ ]:
path_dir = Path(r"\\hunzikerwater.ch\DFSHBT\Daten\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen"
    r"\Messkampagne\Rohdaten Messkampagne")

info_messstellen = {
    'Messstelle_MO_A15': {
        'col_Q': 'Q Input [l/s]',
    },

    'Messstelle_MO_D1A': {
        'col_Q': 'Q Input [l/s]',
    },

    'Messstelle_WE_83A': {
        'col_Q': 'Q Input [l/s]',
    },

    'Messstelle_ZE_204': {
        'col_Q': 'Q Input [l/s]',
    },
}

# Code

In [ ]:
# Read data Messstellen
data_messstellen = {}
for name, values in info_messstellen.items():
    path = path_dir / name
    for file in path.iterdir():
        data = read_messstelle(file, values['col_Q'])
        data_messstellen.update({name: data})  

In [ ]:
# Read Regendaten Möhlin
path_regen = Path(r"\\hunzikerwater.ch\DFSHBT\Daten\2 Projekte\1000-\1300-\1346\1346.33 V-GEP"
    r"\05 Berechnungen Grundlagen\Messkampagne\Rohdaten Messkampagne\Regenmesser_Möhlin"
    r"\History_Data_of_Regenmesser Möhlin_from_2021-07-22 23-59-59 GMT+0200_to_2021-12-02 23-59-59 GMT+0100.csv")

rain_moh = read_regen(path_regen)

In [ ]:
# Read Regendaten Wegenstetten
path_regen = Path(r"\\hunzikerwater.ch\DFSHBT\Daten\2 Projekte\1000-\1300-\1346\1346.33 V-GEP"
    r"\05 Berechnungen Grundlagen\Messkampagne\Rohdaten Messkampagne\Regenmesser_Möhlin"
    r"\History_Data_of_Regenmesser Möhlin_from_2021-07-22 23-59-59 GMT+0200_to_2021-12-02 23-59-59 GMT+0100.csv")

rain_weg = read_regen(path_regen)

In [ ]:
# Read Regendaten Meteoschweiz
id = 1903
date_format = '%d.%m.%Y'
date_from = '29.07.2021'
date_to = '02.12.2021'
rain_ms = saba.get_data(id=id, date_from=date_from, date_to=date_to, date_format=date_format)

In [ ]:
save_dir = path_dir / 'Plots'
if not save_dir.exists():
    save_dir.mkdir()

for name, data in data_messstellen.items():
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,0], name=name), secondary_y=False)
    fig.add_trace(go.Scatter(x=rain_moh.index, y=rain_moh.iloc[:,0], name='Regen Möhlin (Messstelle)'), secondary_y=True)
    fig.add_trace(go.Scatter(x=rain_weg.index, y=rain_weg.iloc[:,0], name='Regen Wegenstetten (Messstelle)'), secondary_y=True)
    fig.add_trace(go.Scatter(x=rain_ms.index, y=rain_ms.iloc[:,0], name='Regen Möhlin (MeteoSchweiz)'), secondary_y=True)

    fig.update_layout(title=name)
    fig.update_xaxes(title='Datum')
    fig.update_yaxes(title='Durchfluss [l/s]', secondary_y=False)
    fig.update_yaxes(title='Regen [mm]', secondary_y=True)

    save_path = save_dir / f"{name}.html"
    fig.write_html(save_path)